In [1]:
"""
In this notebook i want to try the effect of lemmatizing the sentences.
I will use famous imdb sentiment analysis database.
My theory is:
There must be a difference of semantics according to selected sentence parts.
I will train and test method on 3 sets
1)Only adjective and adverbs
2)Only verbs
3)Verbs adjective and adverbs  
in all cases i will exclude nouns
The results I obtained for accuracy of models

1)Only adjective and adverbs    -> Accuracy %78
2)Only verbs                    -> Accuracy %63
3)Verbs adjective and adverbs   -> Accuracy %79

nltk takes so long,so i could not arrange all cell in correct order.
I runned in different order but arranged like this.

"""
from google.colab import files
uploaded = files.upload()

Saving labeledTrainData.tsv to labeledTrainData (1).tsv


In [2]:
import io
import pandas as pd
import nltk
nltk.download('stopwords')
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
 
df = pd.read_csv(io.BytesIO(uploaded['labeledTrainData.tsv']), sep='\t', quoting=3)
 

In [0]:
def clean_review(text):
    # Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
 
    # Strip escaped quotes
    text = text.replace('\\"', '')
 
    # Strip quotes
    text = text.replace('"', '')
 
    return text
df['cleaned_review'] = df['review'].apply(clean_review)

In [0]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
def get_important_features(sentence):
    features = []
    for w in nltk.word_tokenize(sentence):
        pos = get_wordnet_pos(w)
        if pos == wordnet.ADJ or pos == wordnet.VERB or pos == wordnet.ADV :
            features.append( lemmatizer.lemmatize(w, pos) )
    return " ".join(features) 

def get_important_features_verbs(sentence):
    features = []
    for w in nltk.word_tokenize(sentence):
        pos = get_wordnet_pos(w)
        if  pos == wordnet.VERB  :
            features.append( lemmatizer.lemmatize(w, pos) )
    return " ".join(features)     

def get_important_features_adv_adj(sentence):
    features = []
    for w in nltk.word_tokenize(sentence):
        pos = get_wordnet_pos(w)
        if  pos == wordnet.ADJ or  pos == wordnet.ADV   :
            features.append( lemmatizer.lemmatize(w, pos) )
    return " ".join(features)       



In [0]:
def train_validate(mdl,step):
  for i in range(step):
    mdl.fit(padded_train[:-100], y_train[:-100], epochs=3, batch_size=512, verbose=1,
          validation_data=(padded_train[-100:], y_train[-100:]))
    scores = mdl.evaluate(padded_test, y_test, verbose=1)
    print("Accuracy:", scores[1]) # 0.875
    

In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
cleaned_reviews = df['cleaned_review'].values
lemmatized = [get_important_features(review) for review in cleaned_reviews]




In [0]:

lemmatized_verbs = [get_important_features_verbs(review) for review in cleaned_reviews]

In [0]:

lemmatized_adv_adj = [get_important_features_adv_adj(review) for review in cleaned_reviews]

In [0]:
get_important_features("go down 've start listen watch here there watch watch ")
for index,l in enumerate(lemmatized[0:10]):
  print(index,l)

0 go down 've start listen watch here there watch watch again Maybe just get certain be really just maybe make up be guilty be remember go see be originally release have feel also obvious be bad Visually impressive be so remotely anyway then be go find consent make most say make true be really nice actual finally be only so exclude Criminal be psychopathic powerful want dead so bad be rant want know be be supply so maybe just turn whole Also have have come film usually work let alone whole perform complex be be most not then away do give ironically be be truly most talented ever be guilty Well 've give well do n't know be different close know be extremely nice stupid most be not
1 Classic be very entertain obviously go great faithfully classic succeed do so watch appreciate be not predictable come have only Obviously different only criticize more important be entertain be most never enjoy be classic be very entertain make easy be
2 give mutate use fossilize most deadly however high be 

In [0]:
for index,l in enumerate(lemmatized[0:10]):
  print(index,l)

0 go 've start listen watch watch watch get certain be make be guilty be remember go see be release have feel obvious be bad impressive be be go find consent make most say make true be nice actual be exclude Criminal be psychopathic powerful want dead bad be rant want know be be supply turn whole have have come film work let whole perform complex be be most do give be be most talented be guilty 've give do know be different close know be nice stupid most be
1 Classic be entertain go great classic succeed do watch appreciate be predictable come have different criticize important be entertain be most enjoy be classic be entertain make easy be
2 give mutate use fossilize most high be open stalk restrict be attack large pre-historical be big carnivorous be real be astound be stalk run most third dangerous slow full appear special provide excite stir be be make .Middling react become give physical dodge run dangle ridiculous final small realistic Other be follow much fill be direct take man

In [0]:
df['lemmatized'] = lemmatized

In [0]:

df['lemmatized_verbs'] = lemmatized_verbs

In [25]:
df['lemmatized_adv_adj'] = lemmatized_adv_adj

df.head(10)

,id,sentiment,review,cleaned_review,lemmatized,lemmatized_verbs,lemmatized_adv_adj
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",With all this stuff going down at the moment w...,go down 've start listen watch here there watc...,go 've start listen watch watch watch get be m...,down here there again Maybe just certain reall...
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...",The Classic War of the Worlds by Timothy Hines...,Classic be very entertain obviously go great f...,be entertain go succeed do watch appreciate be...,Classic very obviously great faithfully classi...
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...",The film starts with a manager (Nicholas Bell)...,give mutate use fossilize most deadly however ...,give mutate use fossilize be open stalk restri...,most deadly however high savagely large pre-hi...
3,"""3630_4""",0,"""It must be assumed that those who praised thi...",It must be assumed that those who praised this...,be assume praise great film ever do n't somewh...,be assume praise film do do do do appear Cultu...,great ever n't somewhere n't n't n't swan-song...
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...",Superbly trashy and wondrously unpretentious 8...,Superbly wondrously unpretentious somewhat giv...,give 're deal harrow 're be orphan pervert rip...,Superbly wondrously unpretentious somewhat fal...
5,"""8196_8""",1,"""I dont know why people think this is such a b...",I dont know why people think this is such a ba...,know be such bad get pretty good good do not S...,know be get do be be like like see,such bad pretty good good not Sure offensive g...
6,"""7166_2""",0,"""This movie could have been very good, but com...","This movie could have been very good, but come...",have be very good come up short special act ha...,have be come act have look be be have be be li...,very good up short special n't so there more w...
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm...",I watched this video at a friend's house. I'm ...,watch 'm do not have several most not related ...,watch 'm do have man be prepare let have say s...,not several most not related short own instead...
8,"""319_1""",0,"""A friend of mine bought this film for £1, and...","A friend of mine bought this film for £1, and ...",even then be grossly overprice feature big suc...,be overprice feature talented be take hammer t...,even then grossly big such Billy incredibly co...
9,"""8713_10""",1,"""<br /><br />This movie is full of references....",This movie is full of references. Like Mad M...,be full many clear be much more,be be,full many clear much more


In [27]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding
X_train, X_test, y_train, y_test = train_test_split(df['lemmatized_adv_adj'], df['sentiment'], test_size=0.2)
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), lowercase=True, min_df=3, max_df=0.9, max_features=5000)
padded_train,padded_test,vectorizer = arr_to_padded(X_train,X_test)
MAX_SEQ_LENGHT = len(max(padded_train, key=len))

model_adj = Sequential()
model_adj.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model_adj.add(Conv1D(64, 5, activation='relu'))
model_adj.add(MaxPooling1D(5))
model_adj.add(Flatten())
model_adj.add(Dense(units=64, activation='relu'))
model_adj.add(Dense(units=1, activation='sigmoid'))
 
model_adj.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_adj.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 166, 64)           262528    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 162, 64)           20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 32, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                131136    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 414,273
Trainable params: 414,273
Non-trainable params: 0
________________________________________________

In [28]:
train_validate(model_adj,3)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 612us/step - loss: 0.6758 - accuracy: 0.5971 - val_loss: 0.5679 - val_accuracy: 0.7700
Epoch 2/3
19900/19900 [==============================] - 12s 603us/step - loss: 0.4844 - accuracy: 0.7759 - val_loss: 0.3929 - val_accuracy: 0.8700
Epoch 3/3
5000/5000 [==============================] - 1s 250us/step
Accuracy: 0.8015999794006348
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 606us/step - loss: 0.3684 - accuracy: 0.8387 - val_loss: 0.3638 - val_accuracy: 0.8400
Epoch 2/3
19900/19900 [==============================] - 12s 603us/step - loss: 0.3418 - accuracy: 0.8525 - val_loss: 0.3710 - val_accuracy: 0.8400
Epoch 3/3
5000/5000 [==============================] - 1s 272us/step
Accuracy: 0.7973999977111816
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 607us/s

In [29]:
train_validate(model_adj,3)


Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 601us/step - loss: 0.1869 - accuracy: 0.9309 - val_loss: 0.4800 - val_accuracy: 0.8700
Epoch 2/3
19900/19900 [==============================] - 12s 603us/step - loss: 0.1589 - accuracy: 0.9427 - val_loss: 0.5020 - val_accuracy: 0.8700
Epoch 3/3
5000/5000 [==============================] - 1s 243us/step
Accuracy: 0.770799994468689
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 598us/step - loss: 0.1035 - accuracy: 0.9673 - val_loss: 0.6189 - val_accuracy: 0.8300
Epoch 2/3
19900/19900 [==============================] - 12s 600us/step - loss: 0.0838 - accuracy: 0.9728 - val_loss: 0.6771 - val_accuracy: 0.8100
Epoch 3/3
5000/5000 [==============================] - 1s 248us/step
Accuracy: 0.76419997215271
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 12s 599us/step

In [21]:
train_validate(3)

Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 19s 945us/step - loss: 0.2839 - accuracy: 0.8698 - val_loss: 0.7888 - val_accuracy: 0.6500
Epoch 2/3
19900/19900 [==============================] - 19s 934us/step - loss: 0.2489 - accuracy: 0.8871 - val_loss: 0.8686 - val_accuracy: 0.6800
Epoch 3/3
5000/5000 [==============================] - 2s 386us/step
Accuracy: 0.647599995136261
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 19s 944us/step - loss: 0.1959 - accuracy: 0.9092 - val_loss: 1.0046 - val_accuracy: 0.6300
Epoch 2/3
19900/19900 [==============================] - 19s 940us/step - loss: 0.1777 - accuracy: 0.9169 - val_loss: 1.1131 - val_accuracy: 0.6200
Epoch 3/3
5000/5000 [==============================] - 2s 406us/step
Accuracy: 0.6424000263214111
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 19s 939us/st

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['lemmatized'], df['sentiment'], test_size=0.2)

In [0]:
vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), lowercase=True, min_df=3, max_df=0.9, max_features=5000)

In [0]:
def arr_to_padded(arr_train,arr_test):
    vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'), lowercase=True, min_df=3, max_df=0.9, max_features=5000)
    vectorizer.fit_transform(arr_train)
    word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
    tokenizer = vectorizer.build_tokenizer()
    preprocess = vectorizer.build_preprocessor()
    
    indexes_train = []    
    for x in arr_train:
        words = tokenizer(preprocess(x))
        words_indexes = [word2idx[word] for word in words if word in word2idx]
        indexes_train.append(words_indexes)
    MAX_SEQ_LENGHT = len(max(indexes_train, key=len))  
    N_FEATURES = len(vectorizer.get_feature_names())
    padded_train = pad_sequences(indexes_train, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
    
    indexes_test = []    
    for x in arr_test:
        words = tokenizer(preprocess(x))
        words_indexes = [word2idx[word] for word in words if word in word2idx]
        indexes_test.append(words_indexes)
    padded_test = pad_sequences(indexes_test, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)    
    
    return padded_train,padded_test,vectorizer

In [0]:

padded_train,padded_test,vectorizer = arr_to_padded(X_train,X_test)
MAX_SEQ_LENGHT = len(max(padded_train, key=len))  

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding
 
model = Sequential()
model.add(Embedding(len(vectorizer.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 263, 64)           238592    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 259, 64)           20544     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 51, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3264)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                208960    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 468,161
Trainable params: 468,161
Non-trainable params: 0
________________________________________________

In [0]:
train_validate(3)

Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 29s 1ms/step - loss: 0.0857 - accuracy: 0.9767 - val_loss: 0.8459 - val_accuracy: 0.7600
Epoch 2/3
19900/19900 [==============================] - 28s 1ms/step - loss: 0.0678 - accuracy: 0.9833 - val_loss: 0.9348 - val_accuracy: 0.7700
Epoch 3/3
5000/5000 [==============================] - 2s 474us/step
Accuracy: 0.7788000106811523
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 28s 1ms/step - loss: 0.0447 - accuracy: 0.9900 - val_loss: 1.0770 - val_accuracy: 0.7500
Epoch 2/3
19900/19900 [==============================] - 28s 1ms/step - loss: 0.0378 - accuracy: 0.9923 - val_loss: 1.1674 - val_accuracy: 0.7600
Epoch 3/3
5000/5000 [==============================] - 2s 434us/step
Accuracy: 0.7731999754905701
Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 28s 1ms/step - loss

In [0]:
model.fit(padded_train[:-100], y_train[:-100], 
          epochs=3, batch_size=512, verbose=1,
          validation_data=(padded_train[-100:], y_train[-100:]))

Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.3207 - accuracy: 0.8667 - val_loss: 0.4331 - val_accuracy: 0.7600
Epoch 2/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.2877 - accuracy: 0.8839 - val_loss: 0.4774 - val_accuracy: 0.7700
Epoch 3/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.2580 - accuracy: 0.8982 - val_loss: 0.4808 - val_accuracy: 0.7800


In [0]:
scores = model.evaluate(padded_test, y_test, verbose=1)
print("Accuracy:", scores[1]) # 0.875

5000/5000 [==============================] - 2s 355us/step
Accuracy: 0.7865999937057495


In [0]:
model.fit(padded_train[:-100], y_train[:-100], 
          epochs=3, batch_size=512, verbose=1,
          validation_data=(padded_train[-100:], y_train[-100:]))

Train on 19900 samples, validate on 100 samples
Epoch 1/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.2214 - accuracy: 0.9155 - val_loss: 0.5586 - val_accuracy: 0.7600
Epoch 2/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.1863 - accuracy: 0.9340 - val_loss: 0.6178 - val_accuracy: 0.7400
Epoch 3/3
19900/19900 [==============================] - 24s 1ms/step - loss: 0.1512 - accuracy: 0.9498 - val_loss: 0.6668 - val_accuracy: 0.7500


In [0]:
scores = model.evaluate(padded_test, y_test, verbose=1)
print("Accuracy:", scores[1]) # 0.875

5000/5000 [==============================] - 2s 372us/step
Accuracy: 0.7796000242233276
